In [384]:
%matplotlib inline
from __future__ import print_function

from matplotlib import pyplot as plt
import seaborn as sns

import numpy as np
import pandas as pd

%load_ext autoreload
%autoreload 2
 
from utils.nolearn_net import NeuralNet
from utils.data_utils import *
from utils.iterator import BaseBatchIterator
from utils.training_utils import parse_conf_file
from utils.preprocessing import build_pipeline
import importlib
import pickle

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Test 

In [374]:
config = parse_conf_file('/Users/thorey/Documents/project/competition/fognet/models/lstm/model_3/h_25_rg_1e-07_lr_0.001/conf_model.json')

In [375]:
config

{u'architecture': u'build_simple_lstm',
 u'build_ite': u'benchmark',
 u'conf': u'../models/lstm/model_3/conf_model.json',
 u'continue_training': False,
 u'feats': [u'percip_mm',
  u'humidity',
  u'temp',
  u'leafwet450_min',
  u'leafwet460_min',
  u'leafwet_lwscnt',
  u'gusts_ms',
  u'wind_dir',
  u'wind_ms'],
 u'folder': u'/Users/thorey/Documents/project/competition/fognet/models/lstm/model_3/h_25_rg_1e-07_lr_0.001',
 u'grad_clip': 50,
 u'hiddens': 25,
 u'loss_function': u'squared_error',
 u'lr': 0.001,
 u'model': u'lstm',
 u'model_fname': u'/Users/thorey/Documents/project/competition/fognet/models/lstm/model_3/h_25_rg_1e-07_lr_0.001/model.pkl',
 u'model_graph_fname': u'/Users/thorey/Documents/project/competition/fognet/models/lstm/model_3/h_25_rg_1e-07_lr_0.001/model_history.png',
 u'model_history_fname': u'/Users/thorey/Documents/project/competition/fognet/models/lstm/model_3/h_25_rg_1e-07_lr_0.001/model_history.pkl',
 u'model_name': u'h_25_rg_1e-07_lr_0.001',
 u'name': u'micro',
 u

In [381]:
pipeline = build_pipeline(config['pipe_list'], config['pipe_kwargs'])
nb_features, batch_ite_train, batch_ite_val, batch_ite_test, batch_ite_pred = load_data(
        name=config['name'],
        feats=config['feats'],
        build_ite=config['build_ite'],
        pipeline=pipeline)
model = importlib.import_module(
        'model_defs.%s' % config['model'])
builder = getattr(model, config['architecture'])
architecture = builder(D=nb_features, H=config[
        'hiddens'], grad_clip=config['grad_clip'])

Le train is composed by 21 group and 3738 observation
Le val is composed by 8 group and 1112 observation
Le test is composed by 5 group and 948 observation


In [385]:
net = NeuralNet(
        layers=architecture,
        regression=True,
        objective_loss_function=getattr(
            lasagne.objectives, config['loss_function']),
        objective_l2=config['reg'],  # L2 regularization
        update=getattr(lasagne.updates, config['update_rule']),
        update_learning_rate=config['lr'],
        batch_iterator_train=batch_ite_train,
        batch_iterator_test=batch_ite_val,
        verbose=config['verbose'],
        max_epochs=10000,)
net.initialize()
net.load_params_from(config['model_fname'])
with open(config['model_history_fname']) as f:
    net.train_history_ = pickle.load(f)

Loaded parameters to layer 'lstm' (shape 9x25).
Loaded parameters to layer 'lstm' (shape 25x25).
Loaded parameters to layer 'lstm' (shape 25).
Loaded parameters to layer 'lstm' (shape 9x25).
Loaded parameters to layer 'lstm' (shape 25x25).
Loaded parameters to layer 'lstm' (shape 25).
Loaded parameters to layer 'lstm' (shape 9x25).
Loaded parameters to layer 'lstm' (shape 25x25).
Loaded parameters to layer 'lstm' (shape 25).
Loaded parameters to layer 'lstm' (shape 9x25).
Loaded parameters to layer 'lstm' (shape 25x25).
Loaded parameters to layer 'lstm' (shape 25).
Loaded parameters to layer 'lstm' (shape 25).
Loaded parameters to layer 'lstm' (shape 25).
Loaded parameters to layer 'lstm' (shape 25).
Loaded parameters to layer 'lstm' (shape 1x25).
Loaded parameters to layer 'lstm' (shape 1x25).
Loaded parameters to layer 'dense' (shape 25x1).
Loaded parameters to layer 'dense' (shape 1).


In [388]:
net.get_score_whole_set('train'),net.get_score_whole_set('test')

(3.0808924682517103, 5.8183123682635864)

In [389]:
from utils.hook import write_final_score

In [390]:
write_final_score(config,net)

In [395]:
from utils.prediction_utils import *

In [397]:
    final_pred = prediction(net, batch_ite_pred)
    make_submission(config, final_pred)

Will write output to /Users/thorey/Documents/project/competition/fognet/models/lstm/model_3/h_25_rg_1e-07_lr_0.001/submissions_20160322_144424.csv


In [440]:
def conf_generator(config, parameters_grids):

    assert all([f in ['lr', 'reg', 'hiddens'] for f in parameters_grid.keys()])
    product = [x for x in apply(itertools.product, parameters_grid.values())]
    conf_runs = [dict(zip(parameters_grid.keys(), p)) for p in product]
    for conf_run in conf_runs:
        config.update(conf_run)
        yield config

In [441]:
parameters_grid = {'lr':np.arange(1,10,1),
                  'reg':np.arange(1,10,1),
                  'hiddens':np.arange(1,10,1)}

In [435]:
    parameters_grid = {'lr': np.logspace(-7, -1, num=20),
                       'reg': np.logspace(-7, -1, num=20),
                       'hiddens': range(25, 525, 25)}
    confs = conf_generator(config, parameters_grid)

In [437]:
import multiprocessing

In [438]:
multiprocessing.cpu_count()

4

In [434]:
for i,elt in enumerate(confs):
    print(elt['lr'])
    if i>10:
        sys.exit()

2
2
2
2
2
2
3
3
3
3
3
3


SystemExit: 

To exit: use 'exit', 'quit', or Ctrl-D.


In [439]:

from multiprocessing import Pool 

In [449]:
parameters_grid

{'hiddens': array([1, 2, 3, 4, 5, 6, 7, 8, 9]),
 'lr': array([1, 2, 3, 4, 5, 6, 7, 8, 9]),
 'reg': array([1, 2, 3, 4, 5, 6, 7, 8, 9])}

In [453]:
def test(config):
    print(config['lr'],config['reg'],config['hiddens'])

In [ ]:
def conf_generator(config, parameters_grids):

    assert all([f in ['lr', 'reg', 'hiddens'] for f in parameters_grid.keys()])
    product = [x for x in apply(itertools.product, parameters_grid.values())]
    conf_runs = [dict(zip(parameters_grid.keys(), p)) for p in product]
    for conf_run in conf_runs:
        config.update(conf_run)
        yield config

In [459]:
for elt in confs:
    print(elt)

9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 

In [457]:
confs = conf_generator(config,parameters_grid)

In [458]:
pool = Pool()
pool.map(test, confs)
pool.close() 
pool.join()

9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 

In [476]:
    parameters_grid = {'lr': np.logspace(-3, -2, num=2),
                       'reg': [1e-7],
                       'hiddens': [25, 1,50]}

In [523]:
class MyDict(dict):

    def update(self, *args):
        dict.update(self, *args)
        return self

In [546]:
def conf_generator(config, parameters_grids):

    assert all([f in ['lr', 'reg', 'hiddens'] for f in parameters_grid.keys()])
    product = [x for x in apply(itertools.product, parameters_grid.values())]
    conf_runs = [dict(zip(parameters_grid.keys(), p)) for p in product]
    confs = map(lambda d: update_dict(config, d), conf_runs)
    return confs


In [548]:
confs = conf_generator(config,parameters_grid)

In [549]:
for elt in confs:
    print(elt['lr'],elt['hiddens'],elt['reg'])

0.001 25 1e-07
0.01 25 1e-07
0.001 1 1e-07
0.01 1 1e-07
0.001 50 1e-07
0.01 50 1e-07


In [515]:
def worker(elt):
    print(elt['lr'],elt['hiddens'],elt['reg'])

In [550]:
confs = conf_generator(config,parameters_grid)
pool = Pool()
pool.map(worker, ite)
pool.close()
pool.join()

0.001 25 1e-07
0.01 25 1e-07
0.001 1 1e-07
0.01 1 1e-07
0.001 50 1e-07
0.01 50 1e-07


In [494]:
class myDict(dict):
    def update(self, *args):
    	dict.update(self, *args)
    	return self

In [496]:
myconfig = myDict(config)

In [498]:
map(myconfig.update,ite)

[{u'architecture': u'build_simple_lstm',
  u'build_ite': u'benchmark',
  u'conf': u'../models/lstm/model_3/conf_model.json',
  u'continue_training': False,
  u'feats': [u'percip_mm',
   u'humidity',
   u'temp',
   u'leafwet450_min',
   u'leafwet460_min',
   u'leafwet_lwscnt',
   u'gusts_ms',
   u'wind_dir',
   u'wind_ms'],
  u'folder': u'/Users/thorey/Documents/project/competition/fognet/models/lstm/model_3/h_25_rg_1e-07_lr_0.001',
  u'grad_clip': 50,
  u'hiddens': 50,
  u'loss_function': u'squared_error',
  u'lr': 0.01,
  u'model': u'lstm',
  u'model_fname': u'/Users/thorey/Documents/project/competition/fognet/models/lstm/model_3/h_25_rg_1e-07_lr_0.001/model.pkl',
  u'model_graph_fname': u'/Users/thorey/Documents/project/competition/fognet/models/lstm/model_3/h_25_rg_1e-07_lr_0.001/model_history.png',
  u'model_history_fname': u'/Users/thorey/Documents/project/competition/fognet/models/lstm/model_3/h_25_rg_1e-07_lr_0.001/model_history.pkl',
  u'model_name': u'h_25_rg_1e-07_lr_0.001',


In [492]:
def update_dict(config,d):
    config.update(d)
    return config


In [493]:
map(update_dict,ite)

TypeError: update_dict() takes exactly 2 arguments (1 given)

In [501]:
class myDict(dict):

    def update(self, *args):
        dict.update(self, *args)
        return self


def conf_generator(config, parameters_grids):

    config = myDict(config)
    assert all([f in ['lr', 'reg', 'hiddens'] for f in parameters_grid.keys()])
    product = [x for x in apply(itertools.product, parameters_grid.values())]
    conf_runs = [dict(zip(parameters_grid.keys(), p)) for p in product]
    conf_runs = map(dict, map(config.update, conf_runs))
    return conf_runs

In [503]:
confs = conf_generator(config,parameters_grid)

In [504]:
len(confs)

6

In [505]:
pool = Pool()
pool.map(worker, confs)
pool.close()
pool.join()

0.01 50 1e-07
0.01 50 1e-07
0.01 50 1e-07
0.01 50 1e-07
0.01 50 1e-07
0.01 50 1e-07


In [552]:
multiprocessing.cpu_count()

4